## Modelowanie i wymiarowanie systemów wielousługowych

In [7]:
import numpy as np
import math
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import seaborn as sns

### Funkcja1: wyznaczanie rozkładu zajętości p[n] wg algorytmu Kaufmana-Robertsa (wzór nr 4)

Oblicza rozkład zajętości w systemie pełnodostępnym z ruchem wielousługowym.

Parametry:
- C      - pojemność systemu (liczba AUs),
- a_list - lista średnich natężeń ruchu [a1, a2, ..., am],
- t_list - lista żądań zasobowych [t1, t2, ..., tm].

Zwraca:

- p - lista prawdopodobieństw p[n] dla n=0..C,
- s - lista wartości pomocniczych s[n] (przed normalizacją).

In [8]:
def kaufman_roberts(
    C: int,
    a_list: List[float],
    t_list: List[int]) -> Tuple[List[float], List[float]]:

    m = len(a_list)
    if len(t_list) != m:
        raise ValueError("ERROR: Długości a_list i t_list muszą być takie same (m klas).")

    # s[0] = 1
    s = [0.0] * (C + 1)
    s[0] = 1.0

    # s[n] = (1/n) * Σ_i ( a_i * t_i * s[n - t_i] )
    for n in range(1, C + 1):
        acc = 0.0
        for i in range(m):
            ti = t_list[i]
            if n - ti >= 0:
                acc += a_list[i] * ti * s[n - ti]
        s[n] = acc / n

    # normalizacja
    norm = sum(s)
    if norm == 0:
        raise ZeroDivisionError("ERROR: Suma s[n] wyszła 0 - coś jest nie tak z parametrami.")

    p = [sn / norm for sn in s]
    return p, s

### Funkcja2: prawdopodobieństwo blokady dla każdej klasy (wzór 5)

Oblicza prawdopodobieństwa blokady E_i dla każdej klasy i.

Parametry:
- p      – rozkład zajętości p[n] dla n=0..C,
- t_list – lista żądań zasobowych [t1, t2, ..., tm].

Zwraca:
- E_list – lista prawdopodobieństw blokady [E1, E2, ..., Em].

In [9]:
def blocking_probabilities(
    p: List[float],
    t_list: List[int]) -> List[float]:

    C = len(p) - 1
    E_list = []

    for ti in t_list:
        # Ei = suma p[n] dla n = C - ti + 1 .. C
        start = max(0, C - ti + 1)
        Ei = sum(p[n] for n in range(start, C + 1))
        E_list.append(Ei)

    return E_list

### Funkcja3: średnia liczb zajętych zasobów przez klasę i w stanie n (wzór 6)

Oblicza y_i(n) – średnią liczbę zajętych AUs przez klasę i w stanie n.

Wzór:

y_i(n) = **(a_i * t_i * p[n - t_i]) / p[n]** jeśli n - t_i >= 0 i p[n] > 0

**0** w p.p.

Parametry:
- p      – lista p[n], n=0..C
- a_list – lista a_i
- t_list – lista t_i

Zwraca:
- słownik: n -> lista [y_1(n), y_2(n), ..., y_m(n)]

In [10]:
def mean_occupied_per_state(
    p: List[float],
    a_list: List[float],
    t_list: List[int]) -> Dict[int, List[float]]:

    C = len(p) - 1
    m = len(a_list)

    result: Dict[int, List[float]] = {}

    for n in range(C + 1):
        y_n = []
        for i in range(m):
            ti = t_list[i]
            ai = a_list[i]

            if n - ti < 0 or p[n] == 0:
                y_i_n = 0.0
            else:
                y_i_n = (ai * ti * p[n - ti]) / p[n]
            y_n.append(y_i_n)

        result[n] = y_n

    return result

### Funkcja4: wyznaczanie a_i (wzór 7)

Wyznacza a_i ze wzoru:
**a_i = a * C / (m * t_i)**

Parametry:
- a_total – ruch oferowany na jednostkę pojemności (a),
- C       – pojemność systemu,
- t_list  – lista t_i.

Zwraca:
- a_list – lista a_i (dla każdej klasy).

In [11]:
def compute_ai_for_total_a(
    a_total: float,
    C: int,
    t_list: List[int]) -> List[float]:
    
    m = len(t_list)
    a_list = []
    for ti in t_list:
        ai = a_total * C / (m * ti)
        a_list.append(ai)
    return a_list

### Funkcja5: Przykładowe użycie - jedno wywołanie dla zadanych parametrów

Przykład: C = 20, dwa strumienie t1=1, t2=3, a = 0.8 Erl / AU

In [12]:
def example_single_run():
    C = 20
    t_list = [1, 3]   # t1, t2
    a_total = 0.8     # np. jeden punkt z zakresu amin..amax

    a_list = compute_ai_for_total_a(a_total, C, t_list)
    p, s = kaufman_roberts(C, a_list, t_list)
    E_list = blocking_probabilities(p, t_list)
    y = mean_occupied_per_state(p, a_list, t_list)

    print("a_total =", a_total)
    print("a_list  =", a_list)
    print("E_list  =", E_list)
    print("Przykładowe y_i(n) dla n=10:", y[10])

### Funkcja6: Przykład pętli po a (amin..amax z astep)

In [13]:
def sweep_over_a(
    amin: float,
    amax: float,
    astep: float,
    C: int,
    t_list: List[int]):
    
    a_values = []
    E_values_per_class = []  # lista [ [E1(a1), E2(a1), ...], [E1(a2), ...], ... ]

    a = amin
    while a <= amax + 1e-9:  # mała poprawka numeryczna, żeby złapać amax
        a_list = compute_ai_for_total_a(a, C, t_list)
        p, s = kaufman_roberts(C, a_list, t_list)
        E_list = blocking_probabilities(p, t_list)

        a_values.append(a)
        E_values_per_class.append(E_list)

        print(f"a={a:.3f}, E={E_list}")
        a += astep

    return a_values, E_values_per_class

In [ ]:
if __name__ == "__main__":
    # uruchomienie przykładu jednorazowego
    example_single_run()

    # uruchomienie pętli po a (parametry możesz zmienić na te z zadania)
    print("\n--- Sweep over a ---")
    sweep_over_a(
        amin=0.2,
        amax=1.3,
        astep=0.1,
        C=20,
        t_list=[1, 3])

a_total = 0.8
a_list  = [8.0, 2.6666666666666665]
E_list  = [0.060922492153821374, 0.206913223030226]
Przykładowe y_i(n) dla n=10: [6.467772814294832, 3.5322271857051692]

--- Sweep over a ---
a=0.200, E=[4.108959782420867e-05, 0.00029033812849145224]
a=0.300, E=[0.0005621803386351051, 0.003199577617943041]
a=0.400, E=[0.0030101426044448873, 0.01468812613838458]
a=0.500, E=[0.009502747310561216, 0.041199620257967744]
a=0.600, E=[0.021503784921276994, 0.08473791433287946]
a=0.700, E=[0.039020593921605236, 0.14194713293440261]
a=0.800, E=[0.06092249215382137, 0.20691322303022597]
a=0.900, E=[0.08568981484745405, 0.2740619276798154]
a=1.000, E=[0.11194514612350381, 0.339443548928289]
a=1.100, E=[0.13865172278929686, 0.4007734173614441]
a=1.200, E=[0.1651118910588361, 0.45700026025119966]
a=1.300, E=[0.19089389657537845, 0.5078477054679766]
